<a href="https://colab.research.google.com/github/ASIF-Mahmud1/Exploration/blob/text-classifier/NLP_Lessons/DiminishingTermsWithoutWatson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# evaluate multinomial logistic regression model
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression


url="https://raw.githubusercontent.com/ASIF-Mahmud1/Exploration/diminishingTerms/DiminishingTerms/dataSet.csv"
from io import StringIO
import string
import pandas as pd
import requests
s=requests.get(url).text

message_data=pd.read_csv(StringIO(s))
message_data['tag'] = message_data['tag'].str.strip()
# message_data['sentence'] = message_data['sentence'].str.strip()
message_data.head()

,id,tag,sentence
0,1,diminishing,I�m no expert but �
1,2,strong,This is what I see
2,3,diminishing,It�s just my opinion �
3,4,strong,It�s my opinion�
4,5,diminishing,Just checking in �


In [ ]:
message_data_copy = message_data['sentence'].copy() 

message_data_copy

0                        I�m no expert but �
1                         This is what I see
2                     It�s just my opinion �
3                           It�s my opinion�
4                         Just checking in �
                       ...                  
71               Sorry for the inconvinience
72    Sorry about the dog in the background�
73                    Sorry if I look tired�
74                     Sorry, I didn�t know 
75             Thank you for the information
Name: sentence, Length: 76, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def text_preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return " ".join(text)

In [ ]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [ ]:
message_data_copy = message_data_copy.apply(text_preprocess)
message_data_copy = message_data_copy.apply(stemmer)    # New Line
# message_data_copy

In [ ]:
vectorizer = TfidfVectorizer("english")
message_mat = vectorizer.fit_transform(message_data_copy)

In [ ]:
message_train, message_test, spam_nospam_train, spam_nospam_test = train_test_split(message_mat, 
                                                        message_data['tag'], random_state=20)

In [ ]:
Spam_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
Spam_model.fit(message_train, spam_nospam_train)
pred = Spam_model.predict(message_test)
print("The prediction is :", pred)

The prediction is : ['diminishing ' 'strong' 'strong' 'strong' 'diminishing ' 'strong'
 'strong' 'strong' 'strong' 'diminishing ' 'diminishing ' 'diminishing '
 'strong' 'strong' 'strong' 'diminishing ' 'strong' 'strong' 'strong']


In [ ]:
dummyEmail = [" I understand "]
dummyEmailMatrix= vectorizer.transform(dummyEmail) 
dummyEmailMatrix
predict= Spam_model.predict(dummyEmailMatrix)
print("The prediction is :", predict)

The prediction is : ['strong']


In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# define the stages of the pipeline
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,
                                                      max_features=1000,
                                                      stop_words=stopwords.words('english') )),
                            ('model',  LogisticRegression(multi_class='multinomial', solver='lbfgs'))])

In [ ]:
# fit the pipeline model with the training data                            
pipeline.fit(message_data_copy,  message_data['tag'])

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=1000,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you...
                                 strip_accents=None, sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 LogisticRegression(

In [ ]:
text = ["I am hopeless at", "Sorry, I got it wrong"]

# predict the label using the pipeline
pipeline.predict(text)

array(['diminishing ', 'diminishing '], dtype=object)

In [ ]:
prob_Of_Each_Class = Spam_model.predict_proba(dummyEmailMatrix)
# summarize the predicted probabilities
print('Predicted Probabilities: %s' % prob_Of_Each_Class[0])

Predicted Probabilities: [0.48960909 0.49283887 0.01755204]
